In [1]:
import numpy as np
import random

Things to try: 
self-unknown?
input cliques, instead of just pairs
distances between voters?
directed pairs of neighbors? e.g. celebrities
different local allocation rules (as keywords in allocate function)
different graphs, sparsities

In [2]:
# each voter is an index in the array

def construct_neighbors(truecompetencies, neighborpairs):
    """Returns direct neighbors of voters given pairs of neighbors (edges in graph)
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        neighborpairs: list of unrepeated, undirected pairs of neighbors; not necessarily ordered
    
    OUTPUTS:
        neighbors: list of direct neighbors for each voter
    """
    n = len(truecompetencies)
    
    if np.sum(np.array(neighborpairs).flatten()>n-1) != 0:
        return("Error: Pair contains nonexistent voter.")
    
    # initialize neighborlist
    neighbors = [[] for i in range(n)]
    
    # add neighbors
    for pair in neighborpairs:
        (p1, p2) = pair
        neighbors[p1].append(p2)
        neighbors[p2].append(p1)

    return neighbors

In [3]:
# perceived competencies are just based on public voting record. extend to voting for representative?
# unless we want to consider if stop watching -- if i vote right i don't look at others?
def prior_competencies(competencies):
    """Initializes Beta(1,1) public competency distribution priors for everyone
    ARGS:
        competencies: list of true competencies (between 0 and 1) for each voter
    OUTPUTS:
        priors: list of Beta parameters for each individual
    """
    n = len(competencies)
    priors = [(1,1) for i in range(n)] # initialize everyone to Beta(1,1)=Unif(0,1) distributions
    return priors

In [4]:
def allocate(truecompetencies, pubcompetencies, neighbors, mechanism="standard"):
    """Returns index of final voter whom each voter allocates their vote to
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        pubcompetencies: list of public competency distributions for each voter, as paramters for Beta dist
        neighbors: list of direct neighbors for each voter
    Options:
        mechanism: local delegation mechanism to use (see below)
    Mechanisms:
        "standard": each voter delegates to their neighbor with highest mean competency.
                    in case of ties, voter delgates to the first in their list
        "truestandard": each voter delegates to their neighbor with highest true competency.
                    in case of ties, voter delgates to the first in their list
                    
        TODO: other (non-deterministic) mechanisms?
    
    OUTPUTS:
        allocation: list of indices, where each value is the index of the sink
                    voter whom that voter delegates to
    """
    
    allocation = [i for i in range(len(truecompetencies))] # initially each voter allocates vote to self
    
    meanpubcompetencies = [j[0]/(j[0]+j[1]) for j in pubcompetencies] # mean of each dist
    
    # for every voter, consider their neighbors
    for i, ineighbors in enumerate(neighbors):
        # i is index of voter being considered, ineighbors is list of indices of i's neighbors
        if mechanism=="standard":
            neighborcompetencies = [meanpubcompetencies[j] for j in ineighbors] 
            bestneighbor = ineighbors[np.argmax(neighborcompetencies)] # picks neighbor with highest competency; if ties, chooses first
            
            if truecompetencies[i] < meanpubcompetencies[bestneighbor]:
                allocation[i] = bestneighbor
                
        elif mechanism=="truestandard":
            neighborcompetencies = [truecompetencies[j] for j in ineighbors]
            bestneighbor = ineighbors[np.argmax(neighborcompetencies)] # picks neighbor with highest competency; if ties, chooses first

            if truecompetencies[i] < truecompetencies[bestneighbor]:
                allocation[i] = bestneighbor
    
    # Here we allocate to the last sink in the chain
    for node in allocation:
        path = [node]
        while allocation[path[-1]] != path[-1]:
            path.append(allocation[path[-1]])
            if path[-1] == path[0]:
                # Not a DAG. Most of our allocations are not.
                path[-1] = random.choice(path[:-1])
                break
        for point_to_sink in path:
            allocation[point_to_sink] = path[-1]
    return allocation

In [5]:
star_truecomps = np.array([.8]+[.6 for i in range(8)]) # true competencies
star_pubcomps = prior_competencies(star_truecomps) # initialize parameters for priors
star_nbpairs = np.array([(0, i) for i in range(1,9)]) # pairs of neighbors
print(star_truecomps, star_pubcomps, star_nbpairs)

[0.8 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6] [(1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)] [[0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]
 [0 7]
 [0 8]]


In [6]:
# test once
star_nbs = construct_neighbors(star_truecomps, star_nbpairs)
print(allocate(star_truecomps, star_pubcomps, star_nbs, mechanism="standard")) # allocate using mean perceived competency
print(allocate(star_truecomps, star_pubcomps, star_nbs, mechanism="truestandard")) # allocate using true competency

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
def update_competencies(initialcompetencies, votes):
    """Returns updated competencies
    ARGS:
        initialcompetencies: list of initial public competency distributions for each voter, as parameters for Beta dist
        votes: list of voting correctness for each voter
        
    OUTPUTS:
        finalcompetencies: list of initial public competency distributions for each voter, as parameters for Beta dist
    """
    finalcompetencies = initialcompetencies.copy()
    for i in range(len(votes)):
        finalcompetencies[i] = (initialcompetencies[i][0] + 1, initialcompetencies[i][1]) if votes[i]==1 else (initialcompetencies[i][0], initialcompetencies[i][1] + 1)
    
    return finalcompetencies

In [8]:
star_pubcomps
testvotes = [1 for i in range(5)] + [0 for i in range(4)]
star_pubcomps2 = update_competencies(star_pubcomps, testvotes)
star_pubcomps2

[(2, 1), (2, 1), (2, 1), (2, 1), (2, 1), (1, 2), (1, 2), (1, 2), (1, 2)]

In [9]:
def simulate_liquid(truecompetencies, neighborpairs, pubcompetencies=None, rounds=100, mechanism="standard"):
    """Returns final public competencies, arrays of nubmer of voters who allocate, nubmer of voters who vote correctly
    ARGS:
        truecompetencies: list of true competencies (between 0 and 1) for each voter
        neighborpairs: list of unrepeated, undirected pairs of neighbors; not necessarily ordered
    Options:
        pubcompetencies: list of public competency distributions for each voter, as parameters for Beta dist
        rounds: # of iterative rounds to vote
        mechanism: local delegation mechanism to use (see allocate function)
        
    OUTPUTS:
        pubcompetencies: list of final public competency distributions for each voter, as parameters for Beta dist
        numallocated: number of voters who allocate their votes in each round
        numcorrect: number of voters who vote correctly in each round
        numnoallocate: control test TODO: delete
    """
    
    n = len(truecompetencies)
    neighbors = construct_neighbors(truecompetencies, neighborpairs)
    
    if pubcompetencies==None:
        pubcompetencies = prior_competencies(truecompetencies) # initialize to Beta(1,1)
    
    numallocated = []
    numcorrect = []
    numnoallocate = [] # can ignore: testing, for comparison
    for t in range(rounds):
        # allocate
        allocation = allocate(truecompetencies, pubcompetencies, neighbors, mechanism="standard")
        # vote
        initialvotes = np.random.binomial(1, truecompetencies)
        allocatedvotes = initialvotes[allocation]
        
        allocated = np.count_nonzero(np.array(allocation)-np.array([i for i in range(len(truecompetencies))])) # number of voters who allocated their vote to someone else
        
        numallocated.append(allocated) # add number of voters who give up their vote
        numcorrect.append(np.sum(allocatedvotes)) # add number correct to correct
        numnoallocate.append(np.sum(initialvotes)) # add initial number correct to noallocate

        # update perceived competencies
        pubcompetencies = update_competencies(pubcompetencies, allocatedvotes) #NOTE: judging everyone by their final (possibly allocated) vote!!!!
    
    propcorrect = np.sum(numcorrect>np.ceil((n+1)/2.))/rounds # proportion over time of votes that are "correct" (>1/2 of voters vote correctly)
    propnoallocate = np.sum(numnoallocate>np.ceil((n+1)/2.))/rounds
    print("Proportion correct:", propcorrect, " Proportion noallocate right: ", propnoallocate)
    return pubcompetencies, np.array(numallocated), np.array(numcorrect), np.array(numnoallocate)

In [10]:
for center_competency in [0.0, 0.2, 0.4, 0.6, 0.7, 0.8, 0.9, 1.0]:
    star_truecomps = np.array([center_competency]+[.6 for i in range(8)]) # true competencies
    star_nbpairs = np.array([(0, i) for i in range(1,9)]) # pairs of neighbors
    print("Center competency:", center_competency)
    simulate_liquid(star_truecomps, star_nbpairs, rounds=1000, mechanism="standard")
    # simulate_liquid(star_truecomps, star_nbpairs, rounds=100, mechanism="truestandard")

Center competency: 0.0
Proportion correct: 0.478  Proportion noallocate right:  0.319
Center competency: 0.2
Proportion correct: 0.475  Proportion noallocate right:  0.373
Center competency: 0.4
Proportion correct: 0.471  Proportion noallocate right:  0.404
Center competency: 0.6
Proportion correct: 0.565  Proportion noallocate right:  0.491
Center competency: 0.7
Proportion correct: 0.733  Proportion noallocate right:  0.516
Center competency: 0.8
Proportion correct: 0.802  Proportion noallocate right:  0.53
Center competency: 0.9
Proportion correct: 0.899  Proportion noallocate right:  0.567
Center competency: 1.0
Proportion correct: 1.0  Proportion noallocate right:  0.61


In [ ]:
|